In [ ]:
import os
import sys
import time
import asyncio
import json
import nest_asyncio
import subprocess
import shutil
from google.colab import drive
from fastapi import FastAPI, HTTPException, Request
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn
import libtorrent as lt
# 0. CLEANUP
print("Cleaning up old processes...")
!fuser -k 8000/tcp > /dev/null 2>&1
!pkill -f localtunnel > /dev/null 2>&1
!pkill -f serveo > /dev/null 2>&1
!rm -f nohup.out
print("Cleanup done.")
# 1. Install Dependencies
print("Installing dependencies...")
!apt-get install python3-libtorrent -y > /dev/null
!pip install fastapi uvicorn nest_asyncio > /dev/null
print("Dependencies installed.")
# 2. Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')
DRIVE_PATH = "/content/drive/MyDrive/TorrentDownloads"
TEMP_PATH = "/content/temp_downloads"
for path in [DRIVE_PATH, TEMP_PATH]:
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Created directory: {path}")
# 3. Initialize Libtorrent Session
ses = lt.session()
ses.listen_on(6881, 6891)
settings = {
    'user_agent': 'qBittorrent/4.6.0', # Updated UA
    'download_rate_limit': 0, 
    'upload_rate_limit': 0,
    'active_downloads': 50,
    'active_seeds': 20,
    'connections_limit': 2000,
    'cache_size': 65536, # ~1GB Cache
    'cache_expiry': 600,
    'mixed_mode_algorithm': lt.bandwidth_mixed_algo_t.prefer_tcp,
    'announce_to_all_trackers': True,
    'announce_to_all_tiers': True,
}
try:
    ses.apply_settings(settings)
except Exception as e:
    pass
ses.start_dht()
ses.start_lsd()
ses.start_upnp()
ses.start_natpmp()
# Boost DHT with common nodes
ses.add_dht_router("router.bittorrent.com", 6881)
ses.add_dht_router("router.utorrent.com", 6881)
ses.add_dht_router("router.bitcomet.com", 6881)
ses.add_dht_router("dht.transmissionbt.com", 6881)
print("Libtorrent session started (V8 Ultimate Trackers).")
# MASSIVE TRACKER LIST (The "Must Have" List)
TRACKERS = [
    "udp://tracker.opentrackr.org:1337/announce",
    "udp://9.rarbg.com:2810/announce",
    "udp://tracker.openbittorrent.com:80/announce",
    "udp://opentracker.i2p.rocks:6969/announce",
    "udp://tracker.internetwarriors.net:1337/announce",
    "udp://tracker.leechers-paradise.org:6969/announce",
    "udp://coppersurfer.tk:6969/announce",
    "udp://tracker.zer0day.to:1337/announce",
    "udp://tracker.coppersurfer.tk:6969/announce",
    "udp://open.demonii.com:1337/announce",
    "udp://tracker.torrent.eu.org:451/announce",
    "udp://tracker.srv00.com:6969/announce",
    "udp://tracker.dump.cl:6969/announce",
    "udp://tracker.sbsub.com:2710/announce",
    "udp://tracker.cyberia.is:6969/announce",
    "udp://tracker.moeking.me:6969/announce",
    "udp://open.stealth.si:80/announce",
    "http://tracker.openbittorrent.com:80/announce",
    "udp://exodus.desync.com:6969/announce",
    "udp://tracker.tiny-vps.com:6969/announce",
    "udp://tracker.theoks.net:6969/announce",
    "udp://retracker.lanta-net.ru:2710/announce",
    "udp://bt.xxx-tracker.com:2710/announce",
    "udp://tracker.bgm.rur:6969/announce",
    "udp://tracker.ahoy.ooo:6969/announce",
    "udp://open.dstud.io:6969/announce",
]
# 4. Background Task: Monitor & Move
moving_torrents = set()
start_times = {}
async def monitor_torrents():
    print("Background Monitor Started...")
    while True:
        handles = ses.get_torrents()
        for h in handles:
            s = h.status()
            info_hash = str(h.info_hash())
            
            # Auto-record start time for existing/restored torrents if missing
            if info_hash not in start_times:
                 start_times[info_hash] = time.time()
            if s.is_finished and info_hash not in moving_torrents:
                print(f"Torrent finished: {s.name}. Moving to Drive...")
                moving_torrents.add(info_hash)
                asyncio.create_task(move_to_drive(h, s.name, info_hash))
                
        await asyncio.sleep(5)
async def move_to_drive(handle, name, info_hash):
    try:
        handle.pause()
        src = os.path.join(TEMP_PATH, name)
        dst = os.path.join(DRIVE_PATH, name)
        
        print(f"Moving {src} -> {dst}")
        
        if os.path.exists(src):
            await asyncio.to_thread(shutil.move, src, dst)
            print(f"Successfully moved: {name}")
            ses.remove_torrent(handle)
        else:
             # Sometimes it's a folder, ensure dir exists
            if not os.path.exists(os.path.dirname(dst)):
                os.makedirs(os.path.dirname(dst))
            await asyncio.to_thread(shutil.move, src, dst)
            print(f"Successfully moved: {name}")
            ses.remove_torrent(handle)
            
    except Exception as e:
        print(f"Error moving {name}: {e}")
    finally:
        pass
# 5. FastAPI App
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
nest_asyncio.apply()
class MagnetRequest(BaseModel):
    link: str
class ControlRequest(BaseModel):
    info_hash: str
    action: str
@app.get("/")
def read_root():
    return {"status": "running", "service": "Colab Torrent Backend V9"}
@app.post("/control")
def control_torrent(req: ControlRequest):
    handles = ses.get_torrents()
    target = None
    for h in handles:
        if str(h.info_hash()) == req.info_hash:
            target = h
            break
            
    if not target and req.action != 'delete_force': 
        raise HTTPException(status_code=404, detail="Torrent not found")
        
    if req.action == 'pause':
        target.pause()
    elif req.action == 'resume':
        target.resume()
    elif req.action == 'delete':
        ses.remove_torrent(target, 1)
        if req.info_hash in moving_torrents:
            moving_torrents.remove(req.info_hash)
        if req.info_hash in start_times:
            del start_times[req.info_hash]
            
    return {"status": "ok", "action": req.action}
@app.post("/add")
def add_torrent(req: MagnetRequest):
    try:
        magnet = req.link
        if "xt=urn:btih:" in magnet:
             for tr in TRACKERS:
                 magnet += f"&tr={tr}"
        params = {
            'save_path': TEMP_PATH,
            'storage_mode': lt.storage_mode_t(1),
        }
        handle = lt.add_magnet_uri(ses, magnet, params)
        info_hash = str(handle.info_hash())
        start_times[info_hash] = time.time()
        print(f"Added magnet link...")
        
        return {"status": "success", "info_hash": info_hash}
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))
@app.get("/torrents")
def get_torrents():
    torrents = []
    handles = ses.get_torrents()
    for h in handles:
        s = h.status()
        info_hash = str(h.info_hash())
        
        state = "downloading"
        if s.is_finished:
            if info_hash in moving_torrents:
                state = "moving_to_drive" 
            elif h not in handles:
                state = "completed"
            else:
                state = "finished_waiting"
        elif s.state == lt.torrent_status.checking_files:
             state = "checking"
        elif s.state == lt.torrent_status.downloading_metadata:
             state = "fetching_metadata"
        elif s.paused:
             state = "paused"
             
        elapsed = 0
        if info_hash in start_times:
             elapsed = time.time() - start_times[info_hash]
             
        torrents.append({
            "name": s.name if s.name else "Fetching Metadata...",
            "progress": s.progress * 100,
            "state": state,
            "download_rate": s.download_rate, 
            "upload_rate": s.upload_rate,
            "total_done": s.total_done,
            "total_size": s.total_wanted,
            "peers": s.num_peers,
            "info_hash": info_hash,
            "elapsed": elapsed
        })
    return torrents
# 6. Start Server & Tunnel (Serveo)
async def start_tunnel():
    print("Starting Serveo Tunnel...")
    process = await asyncio.create_subprocess_exec(
        'ssh', '-o', 'StrictHostKeyChecking=no', '-R', '80:localhost:8000', 'serveo.net',
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )
    while True:
        line = await process.stdout.readline()
        if not line: break
        decoded_line = line.decode().strip()
        print(f"Serveo: {decoded_line}")
        if "Forwarding HTTP traffic from" in decoded_line:
            url = decoded_line.split('from ')[1]
            print(f"\n--> COPY THIS URL: {url} <--\n")
if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.create_task(start_tunnel())
    loop.create_task(monitor_torrents())
    
    config = uvicorn.Config(app, host="0.0.0.0", port=8000)
    server = uvicorn.Server(config)
    await server.serve()
